JupyterLab allows writing SQL query directly in-cell, thanks to the `%sparksql` magic command (use two percent signs `%%sparksql` to span code in multiple lines). An amazing feature is that PySpark can also interacts with this enviroment. This means all local files can be read as Hive tables.

In [1]:
import findspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').enableHiveSupport().getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 80)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
%load_ext sparksql_magic

In [3]:
%config SparkSql.limit=20

# 1. Managing tables

## 1.1. Creating tables

#### Manually creating tables

In [7]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_product
STORED AS PARQUET
LOCATION 'spark_db/tbl_product'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    ('Laptop', 1000, 15),
    ('Mouse', 20, 100),
    ('Headphone', 50, 50),
    ('USB', NULL, 100)
AS (product, price, stock)

#### Metadata

|Statement|Usage|
|:--|:--|
|`DESC <table_name>`|Show columns and comments of a table|
|`DESC FORMATTED <table_name>`|Show detailed information of a table|
|`SHOW CREATE TABLE <table_name>`|Get the script that created the table|
|`DROP TABLE IF EXISTS <table_name>`|Drop a table|
|`SHOW DATABASES`|Show all available databases|
|`DESC DATABASE EXTENDED <database_name>`|Show information about a database|
|`USE <database_name>`|Enter a specific database|
|`SHOW TABLES`|Show all available tables and view|
|`SHOW TABLES LIKE <pattern>`| Show all tables having a specific pattern in their name|

In [8]:
%%sparksql

DESC tbl_product

col_name,data_type,comment
product,string,null
price,int,null
stock,int,null


In [9]:
%%sparksql

SHOW TABLES LIKE '*product*'

database,tableName,isTemporary
default,tbl_product,False


In [10]:
%%sparksql

ALTER TABLE tbl_product SET TBLPROPERTIES('external'='false', 'auto.purge'='true')

In [11]:
%%sparksql

DROP TABLE IF EXISTS tbl_product

#### Partitioning
A unique feature of SparkSQL is organizing tables in partitions, which helps achieve more parallelism. A categorical column or two may be used as partition columns. Data can be inserted to a partition using `INSERT INTO TABLE` or `INSERT OVERWRITE TABLE`.

In [10]:
%%sparksql

CREATE TABLE IF NOT EXISTS tbl_product (
    product STRING COMMENT 'name of product',
    price INT COMMENT 'price of product',
    stock INT COMMENT 'number of products left'
)
PARTITIONED BY (day STRING COMMENT 'day', hour STRING COMMENT 'hour')
STORED AS PARQUET
LOCATION 'spark_db/tbl_product'
TBLPROPERTIES ('parquet.compression'='snappy')

In [11]:
%%sparksql

INSERT OVERWRITE TABLE tbl_product
PARTITION (day=20210725, hour=14)
VALUES
    ('Laptop', 1000, 25),
    ('Mouse', 30, 100),
    ('Headphone', 50, 50)

In [12]:
%%sparksql

INSERT INTO TABLE tbl_product
PARTITION (day=20210725, hour=21)
VALUES
    ('Laptop', 1000, 20),
    ('Mouse', 20, 97),
    ('Headphone', 65, 12)

In [13]:
%%sparksql

SHOW PARTITIONS tbl_product

partition
day=20210725/hour=14
day=20210725/hour=21


In [14]:
%%sparksql

ALTER TABLE tbl_product DROP IF EXISTS PARTITION (day=20210725)

## 1.2. Importing local files

In [9]:
df = spark.read.csv('data/youtube_trending.csv', header=True, inferSchema=True)

df\
    .write.format('parquet')\
    .option('compression', 'snappy')\
    .mode('overwrite').saveAsTable('tbl_youtube')

In [10]:
%%sparksql

SELECT * FROM tbl_youtube LIMIT 5

video_id,trending_date,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled
2kyS6SvSYSE,2017-11-14,CaseyNeistat,22,2017-11-14 00:13:01,748374,57527,2966,15954,False,False
1ZAPwfrtAFY,2017-11-14,LastWeekTonight,24,2017-11-13 14:30:00,2418783,97185,6146,12703,False,False
5qpjK5DgCt4,2017-11-14,Rudy Mancuso,23,2017-11-13 02:05:24,3191434,146033,5339,8181,False,False
puqaWrEC7tY,2017-11-14,Good Mythical Morning,24,2017-11-13 18:00:04,343168,10172,666,2146,False,False
d380meD0W0M,2017-11-14,nigahiga,24,2017-11-13 01:01:41,2095731,132235,1989,17518,False,False


## 1.3. Hive data types

|Data type|Types|Range
|:--|:--|:--
|Numeric|`TINYINT`|1-byte signed integer, from -128 to 127
||`SMALLINT`|2-byte signed integer, from -32,768 to 32,767
||`INTEGER`|4-byte signed integer, from -2,147,483,648 to 2,147,483,647
||`BIGINT`|8-byte signed integer, from -9,223,372,036,854,775,808 to 9,223,372,036,854,775,807
||`FLOAT`|4-byte single precision floating point number
||`DOUBLE`|8-byte double precision floating point number
||`DECIMAL`|a precision of 38 digits
||`NUMERIC`|same as `DECIMAL`, starting with Hive 3.0.0
|String|`VARCHAR`|Only available starting with Hive 0.12.0
||`CHAR`|Only available starting with Hive 0.13.0
|Datetime|`TIMESTAMP`|
||`DATE`|
||`INTERVAL`|
|Misc|`BOOLEAN`|
||`BINARY`|
|Complex|`ARRAY`|
||`MAP`|
||`STRUCT`|
||`UNIONTYPE`|

In [13]:
%%sparksql

SELECT * FROM tbl_youtube LIMIT 3

video_id,trending_date,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled
2kyS6SvSYSE,2017-11-14,CaseyNeistat,22,2017-11-14 00:13:01,748374,57527,2966,15954,False,False
1ZAPwfrtAFY,2017-11-14,LastWeekTonight,24,2017-11-13 14:30:00,2418783,97185,6146,12703,False,False
5qpjK5DgCt4,2017-11-14,Rudy Mancuso,23,2017-11-13 02:05:24,3191434,146033,5339,8181,False,False


In [11]:
%%sparksql

DESC tbl_youtube

col_name,data_type,comment
video_id,string,null
trending_date,string,null
channel_title,string,null
category_id,int,null
publish_time,timestamp,null
views,int,null
likes,int,null
dislikes,int,null
comment_count,int,null
comments_disabled,boolean,null


In Hive, using `CAST` function to convert between data types. 

In [34]:
%%sparksql

SELECT  category_id, views, comments_disabled,
        CAST(category_id AS FLOAT) AS float_cat_id,
        CAST(views AS BIGINT) AS bigint_views,
        CAST(trending_date AS DATE) AS date,
        CAST(comments_disabled AS INT) AS int_comments_disabled
FROM tbl_youtube LIMIT 3

category_id,views,comments_disabled,float_cat_id,bigint_views,date,int_comments_disabled
22,748374,False,22.0,748374,2017-11-14,0
24,2418783,False,24.0,2418783,2017-11-14,0
23,3191434,False,23.0,3191434,2017-11-14,0


With datetime, we can user both `CAST` or `DATE`/`TO_DATE` to convert string to datetime. Different between date and to_date is `date` can only user with format *yyyy-MM-dd* but `to_date` can add format parameter to fit with the input

*All the datetime functions in hive:* [Datetime functions](https://sparkbyexamples.com/apache-hive/hive-date-and-timestamp-functions-examples/)

In [87]:
%%sparksql

SELECT  CAST('2021-08-10' AS DATE) date,
        DATE('2021/08/10') date1,
        TO_DATE('2021/08/10', 'yyyy/MM/dd') as date2,
        CAST(date_format(current_date(),'yyyy-MM-dd') as date) today,
        DATE(now()) as today1

date,date1,date2,today,today1
2021-08-10,null,2021-08-10,2021-11-05,2021-11-05


In [59]:
%%sparksql

SELECT  publish_time,
        UNIX_TIMESTAMP(publish_time) AS unix_publish_time,
        FROM_UNIXTIME(UNIX_TIMESTAMP(publish_time),'yyyy/MM/dd HH.mm.ss') AS convert_publish_time
FROM tbl_youtube LIMIT 3

publish_time,unix_publish_time,convert_publish_time
2017-11-14 00:13:01,1510593181,2017/11/14 00.13.01
2017-11-13 14:30:00,1510558200,2017/11/13 14.30.00
2017-11-13 02:05:24,1510513524,2017/11/13 02.05.24


In [83]:
%%sparksql

SELECT  current_date() + interval '1' day AS tomorrow,
        current_date() + interval '2' days AS two_days_ago,
        current_date() + interval '1' week AS next_week,
        current_date() - interval '2' months AS two_months_ago,
        current_timestamp() - interval '2' hours AS two_hours_ago

tomorrow,two_days_ago,next_week,two_months_ago,two_hours_ago
2021-11-06,2021-11-07,2021-11-12,2021-09-05,2021-11-05 08:19:37.611000


In [107]:
%%sparksql

SELECT  publish_time,
        DATE(publish_time) AS publish_date,
        YEAR(publish_time) AS publish_year,
        MONTH(publish_time) AS publish_month,
        WEEKOFYEAR(publish_time) AS publish_week,
        QUARTER(publish_time) AS publish_quarter,
        HOUR(publish_time) AS publish_hour,
        TRUNC(CAST(publish_time AS STRING), 'MM') AS publish_firstday_of_month,
        EXTRACT(dayofweek from publish_time) AS publish_dayofweek,
        DATE_FORMAT(publish_time, 'dd.MM.yyyy') AS publish_date_format

FROM tbl_youtube
LIMIT 3

publish_time,publish_date,publish_year,publish_month,publish_week,publish_quarter,publish_hour,publish_firstday_of_month,publish_dayofweek,publish_date_format
2017-11-14 00:13:01,2017-11-14,2017,11,46,4,0,2017-11-01,3,14.11.2017
2017-11-13 14:30:00,2017-11-13,2017,11,46,4,14,2017-11-01,2,13.11.2017
2017-11-13 02:05:24,2017-11-13,2017,11,46,4,2,2017-11-01,2,13.11.2017


In [106]:
%%sparksql

SELECT  publish_time, trending_date,
        DATE_ADD(publish_time, 2) AS date_add,
        DATEDIFF(trending_date, publish_time) AS date_diff
FROM tbl_youtube
LIMIT 3

publish_time,trending_date,date_add,date_diff
2017-11-14 00:13:01,2017-11-14,2017-11-16,0
2017-11-13 14:30:00,2017-11-14,2017-11-15,1
2017-11-13 02:05:24,2017-11-14,2017-11-15,1


## 1.4. Aliases

Alias has been used when name of column or table is too long to use it mutiple times

In [114]:
df = spark.read.csv('data/youtube_category.csv', header=True, inferSchema=True)

df\
    .write.format('parquet')\
    .option('compression', 'snappy')\
    .mode('overwrite').saveAsTable('tbl_youtube_category')

In [125]:
%%sparksql

SELECT  category_id AS id,
        category_name AS name
FROM tbl_youtube_category LIMIT 5

id,name
1,Film & Animation
2,Autos & Vehicles
10,Music
15,Pets & Animals
17,Sports


In [127]:
%%sparksql

SELECT  ytb.video_id, 
        ytb.category_id,
        ytb.publish_time,
        ytb.likes, 
        ytb.views, 
        cat.category_name 
FROM tbl_youtube AS ytb
LEFT JOIN tbl_youtube_category AS cat
ON ytb.category_id = cat.category_id
AND ytb.publish_time >= '2017-11-12'
LIMIT 5

video_id,category_id,publish_time,likes,views,category_name
2kyS6SvSYSE,22,2017-11-14 00:13:01,57527,748374,People & Blogs
1ZAPwfrtAFY,24,2017-11-13 14:30:00,97185,2418783,Entertainment
5qpjK5DgCt4,23,2017-11-13 02:05:24,146033,3191434,Comedy
puqaWrEC7tY,24,2017-11-13 18:00:04,10172,343168,Entertainment
d380meD0W0M,24,2017-11-13 01:01:41,132235,2095731,Entertainment


# 2. Data manipulation

## 2.1. Filtering

Filtering in SQL come with `WHERE` statement. With number or datetime, use `>,=,<` as normal, with string type, some commond function are `RLIKE`,`LIKE`,`IN`,`IS NULL`.
When using multi condition with `AND`, `OR`, we should group statement `OR` with brackets to clear the query

In [138]:
df = spark.read.csv('data/supermarket_sales.csv', header=True, inferSchema=True)

df\
    .write.format('parquet')\
    .option('compression', 'snappy')\
    .mode('overwrite').saveAsTable('tbl_supermarket')

In [169]:
%%sparksql

SELECT * FROM tbl_supermarket 
WHERE quantity >= 5 AND quantity <= 9
LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,03/03/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4


In [175]:
%%sparksql

SELECT * FROM tbl_supermarket 
WHERE date > '3/15/2019' 
LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,3/25/2019,18:30,Ewallet,597.73,4.761904762,29.8865,4.1
829-34-3910,A,Yangon,Normal,Female,Health and beauty,71.38,10,35.69,3/29/2019,19:21,Cash,713.8,4.761904762,35.69,5.7
227-03-5010,A,Yangon,Member,Female,Home and lifestyle,52.59,8,21.036,3/22/2019,19:20,Credit card,420.72,4.761904762,21.036,8.5


In [176]:
%%sparksql

SELECT * FROM tbl_supermarket 
WHERE payment = 'Cash' AND rating > 5
LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
529-56-3974,B,Mandalay,Member,Male,Electronic accessories,25.51,4,5.102,03/09/2019,17:03,Cash,102.04,4.761904762,5.102,6.8
829-34-3910,A,Yangon,Normal,Female,Health and beauty,71.38,10,35.69,3/29/2019,19:21,Cash,713.8,4.761904762,35.69,5.7


In [143]:
%%sparksql

SELECT * FROM tbl_supermarket 
WHERE payment = 'Cash' AND (city = 'Yangon' OR city = 'Mandalay')
LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
529-56-3974,B,Mandalay,Member,Male,Electronic accessories,25.51,4,5.102,03/09/2019,17:03,Cash,102.04,4.761904762,5.102,6.8
829-34-3910,A,Yangon,Normal,Female,Health and beauty,71.38,10,35.69,3/29/2019,19:21,Cash,713.8,4.761904762,35.69,5.7
299-46-1805,B,Mandalay,Member,Female,Sports and travel,93.72,6,28.116,1/15/2019,16:19,Cash,562.32,4.761904762,28.116,4.5


In [144]:
%%sparksql

SELECT * FROM tbl_supermarket
WHERE product_line IS NULL

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating


In [151]:
%%sparksql

SELECT * FROM tbl_supermarket
WHERE brand IN ('A','C') AND product_line LIKE '%beauty'
LIMIT 5

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.626,01/10/2019,17:15,Credit card,72.52,4.761904762,3.626,7.2
829-34-3910,A,Yangon,Normal,Female,Health and beauty,71.38,10,35.69,3/29/2019,19:21,Cash,713.8,4.761904762,35.69,5.7
656-95-9349,A,Yangon,Member,Female,Health and beauty,68.93,7,24.1255,03/11/2019,11:03,Credit card,482.51,4.761904762,24.1255,4.6


In [164]:
%%sparksql

SELECT * FROM tbl_supermarket
WHERE brand NOT IN ('A','C') AND product_line RLIKE ('accessories|travel')
LIMIT 5

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
351-62-0822,B,Mandalay,Member,Female,Fashion accessories,14.48,4,2.896,02/06/2019,18:07,Ewallet,57.92,4.761904762,2.896,4.5
529-56-3974,B,Mandalay,Member,Male,Electronic accessories,25.51,4,5.102,03/09/2019,17:03,Cash,102.04,4.761904762,5.102,6.8
299-46-1805,B,Mandalay,Member,Female,Sports and travel,93.72,6,28.116,1/15/2019,16:19,Cash,562.32,4.761904762,28.116,4.5
649-29-6775,B,Mandalay,Normal,Male,Fashion accessories,33.52,1,1.676,02/08/2019,15:31,Cash,33.52,4.761904762,1.676,6.7
871-79-8483,B,Mandalay,Normal,Male,Fashion accessories,94.13,5,23.5325,2/25/2019,19:39,Credit card,470.65,4.761904762,23.5325,4.8


## 2.2. Aggregating

```sql
GROUP BY: COUNT(*), COUNT(DISTINCT), COUNT(CASE WHEN), SUM, SUM(DISTINCT), AVG, AVG(DISTINCT), MIN, MAX, VAR_POP, STDDEV_POP, PERCENTILE
```

*Reference: [Apache Hive - Aggregate functions](https://cwiki.apache.org/confluence/display/hive/languagemanual+udf#LanguageManualUDF-Built-inAggregateFunctions(UDAF))*

In [178]:
%%sparksql

SELECT * from tbl_supermarket LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,03/03/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4


In [186]:
%%sparksql

SELECT COUNT(*) FROM tbl_supermarket

count(1)
1000


In [185]:
%%sparksql

SELECT  brand, 
        ROUND(SUM(profit),2) total_profit, 
        COUNT(invoice_id) total_invoice,
        COUNT(DISTINCT product_line) num_product_line,
        AVG(unit_price) avg_price,
        MIN(tax) min_tax,
        MAX(tax) max_tax
FROM tbl_supermarket 
GROUP BY brand

brand,total_profit,total_invoice,num_product_line,avg_price,min_tax,max_tax
B,5057.03,332,6,55.65927710843373,0.8875,48.69
C,5265.18,328,6,56.60902439024389,0.5085,49.65
A,5057.16,340,6,54.780852941176484,0.6045,49.49


In [189]:
%%sparksql

SELECT  brand, city, 
        SUM(CASE WHEN customer_type = 'Member' THEN profit ELSE 0 END) member_profit,
        SUM(CASE WHEN customer_type = 'Normal' THEN profit ELSE 0 END) normal_profit
FROM tbl_supermarket
GROUP BY brand, city

brand,city,member_profit,normal_profit
B,Mandalay,2557.366000000001,2499.666
C,Naypyitaw,2708.6325000000006,2556.5439999999994
A,Yangon,2554.165499999999,2502.9950000000017


In [206]:
%%sparksql

SELECT  product_line, 
        ROUND(VAR_POP(unit_price),2) price_var_pop, 
        ROUND(VARIANCE(unit_price),2) price_var_samp,
        ROUND(STDDEV_POP(unit_price),2) price_std,
        ROUND(PERCENTILE(unit_price,0.5),2) price_median
FROM tbl_supermarket
GROUP BY product_line

product_line,price_var_pop,price_var_samp,price_std,price_median
Home and lifestyle,682.51,686.8,26.12,55.59
Fashion accessories,707.1,711.09,26.59,55.09
Health and beauty,689.91,694.48,26.27,57.41
Electronic accessories,711.25,715.46,26.67,51.5
Food and beverages,686.16,690.13,26.19,54.0
Sports and travel,719.52,723.88,26.82,58.65


## 2.3. Window functions

- Windows functions: ROW_NUMBER, RANK, DENSE_RANK, NTILE,... (same as in pyspark)
- Special uses of SUM, COUNT (with or without PARTITION BY)
- ROW|RANGE BETWEEN, UNBOUNDED, PRECEDING, CURRENT_ROW,...

*Reference: [Apache Hive - Window functions](https://cwiki.apache.org/confluence/display/hive/languagemanual+windowingandanalytics)*

In [239]:
%%sparksql

SELECT * FROM tbl_supermarket LIMIT 3

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,01/05/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,03/08/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,03/03/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4


In [236]:
%%sparksql

SELECT *, 
    ROW_NUMBER() OVER(PARTITION BY(brand) ORDER BY(total_profit) DESC) AS row_num,
    RANK() OVER(ORDER BY(total_profit) DESC) AS rank,
    DENSE_RANK() OVER(ORDER BY(total_profit) DESC) AS dense_rank,
    NTILE(4) OVER(ORDER BY(total_profit) DESC) AS ntile
FROM
(SELECT brand, product_line, SUM(profit) total_profit 
FROM tbl_supermarket
GROUP BY brand, product_line)

brand,product_line,total_profit,row_num,rank,dense_rank,ntile
A,Home and lifestyle,1067.4854999999998,1,2,2,1
A,Sports and travel,922.5095,2,6,6,2
A,Electronic accessories,872.2435000000002,3,8,8,2
A,Food and beverages,817.2905,4,10,10,2
A,Fashion accessories,777.7385000000002,5,14,14,3
A,Health and beauty,599.893,6,18,18,4
B,Sports and travel,951.8189999999997,1,4,4,1
B,Health and beauty,951.4599999999999,2,5,5,1
B,Home and lifestyle,835.6745000000001,3,9,9,2
B,Electronic accessories,811.9735000000001,4,11,11,3


In [260]:
%%sparksql

SELECT  invoice_id, brand, city, 
        COUNT(invoice_id) OVER(PARTITION BY (city)) AS total_invoice,
        SUM(quantity) OVER(PARTITION BY (city)) AS total_quantity
FROM tbl_supermarket
LIMIT 4

invoice_id,brand,city,total_invoice,total_quantity
226-31-3081,C,Naypyitaw,328,1831
699-14-3026,C,Naypyitaw,328,1831
315-22-5665,C,Naypyitaw,328,1831
300-71-4605,C,Naypyitaw,328,1831


In [261]:
%%sparksql

SELECT city, COUNT(invoice_id) AS num_invoice
FROM tbl_supermarket
GROUP BY  city

city,num_invoice
Naypyitaw,328
Mandalay,332
Yangon,340


In [265]:
%%sparksql

SELECT *, 
SUM(quantity) OVER (PARTITION BY brand ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as test
FROM tbl_supermarket
LIMIT 5

invoice_id,brand,city,customer_type,gender,product_line,unit_price,quantity,tax,date,time,payment,cost,gross_margin_percentage,profit,rating,test
891-01-7034,B,Mandalay,Normal,Female,Electronic accessories,74.71,6,22.413,01/01/2019,19:07,Cash,448.26,4.761904762,22.413,6.7,6
770-42-8960,B,Mandalay,Normal,Male,Food and beverages,21.12,8,8.448,01/01/2019,19:31,Cash,168.96,4.761904762,8.448,6.3,14
746-04-1077,B,Mandalay,Member,Female,Food and beverages,84.63,10,42.315,01/01/2019,11:36,Credit card,846.3,4.761904762,42.315,9.0,24
244-08-0162,B,Mandalay,Normal,Female,Health and beauty,34.21,10,17.105,01/02/2019,13:00,Cash,342.1,4.761904762,17.105,5.1,34
198-84-7132,B,Mandalay,Member,Male,Fashion accessories,40.61,9,18.2745,01/02/2019,13:40,Cash,365.49,4.761904762,18.2745,7.0,43


## 2.4. Gathering data
- `JOIN`: cross, left, right, inner, outer
- `UNION`: all

In [10]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_user
STORED AS PARQUET
LOCATION 'tbl_user'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    (1, 'Jane', NULL),
    (2, 'Alex', 25),
    (3, 'Mary', 21),
    (4, NULL, 20)
AS (userid, name, age)

In [30]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_sale
STORED AS PARQUET
LOCATION 'tbl_sale'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    (1, 1000, 2, 5),
    (2, 2560, 5, 10),
    (3, 1400, 1, 15),
    (5, 3214, 3, 0)
AS (userid, sale, quantity, shipping_fee)

In [37]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_user_append
STORED AS PARQUET
LOCATION 'tbl_user_append'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    (5, 'Adam', 26),
    (6, 'Kayn', 25),
    (7, 'Kelly', 19),
    (8, 'Muoas', 20)
AS (userid, name, age)

In [31]:
%%sparksql

SELECT u.*, s.* FROM tbl_user u
LEFT JOIN tbl_sale s
ON u.userid = s.userid

userid,name,age,userid,sale,quantity,shipping_fee
3,Mary,21,3,1400,1,15
2,Alex,25,2,2560,5,10
1,Jane,null,1,1000,2,5
4,null,20,null,null,null,null


In [32]:
%%sparksql

SELECT u.*, s.* FROM tbl_user u
INNER JOIN tbl_sale s
ON u.userid = s.userid

userid,name,age,userid,sale,quantity,shipping_fee
3,Mary,21,3,1400,1,15
1,Jane,null,1,1000,2,5
2,Alex,25,2,2560,5,10


In [35]:
%%sparksql

SELECT u.*, s.* FROM tbl_user u
FULL OUTER JOIN tbl_sale s
ON u.userid = s.userid

userid,name,age,userid,sale,quantity,shipping_fee
1,Jane,null,1,1000,2,5
3,Mary,21,3,1400,1,15
null,null,null,5,3214,3,0
4,null,20,null,null,null,null
2,Alex,25,2,2560,5,10


In [40]:
%%sparksql

SELECT * FROM tbl_user 
UNION (SELECT * FROM tbl_user_append) 
ORDER BY userid

userid,name,age
1,Jane,null
2,Alex,25
3,Mary,21
4,null,20
5,Adam,26
6,Kayn,25
7,Kelly,19
8,Muoas,20


## 2.5. Order of execution
```sql
FROM -> JOIN [ON] -> WHERE -> GROUP BY -> HAVING -> SELECT -> ORDER BY -> LIMIT
```

In [53]:
%%sparksql

SELECT  category_name, 
        SUM(likes) total_like, 
        SUM(views) total_view 
FROM
    (SELECT  ytb.video_id, 
            ytb.category_id,
            ytb.publish_time,
            ytb.likes, 
            ytb.views, 
            cat.category_name 
    FROM tbl_youtube AS ytb
    LEFT JOIN tbl_youtube_category AS cat
    ON ytb.category_id = cat.category_id
    WHERE ytb.publish_time >= '2017-11-12')
    
GROUP BY category_name
HAVING total_like > 5e6
ORDER BY total_like DESC

category_name,total_like,total_view
Music,178960460,3943074543
Entertainment,119595586,3913251602
Comedy,50180357,972703458
People & Blogs,25302630,590243343
How to & Style,24267555,694417305
Science & Technology,13931612,476338569
Film & Animation,11891589,574027383
Sports,7067350,340256954
Education,6078265,146637269


## 2.6. Functions
```sql
PI, RAND, LOG, SQRT, POW/POWER, CONCAT, CONCAT_WS, NVL, NVL2, REGEXP_REPLACE, REGEXP_EXTRACT, SPLIT, GREATEST, LEAST, LOWER/UPPER, LENGTH, NULLIF, LPAD/RPAD, LTRIM/RTRIM/TRIM, SUBSTR/SUBSTRING, CASE WHEN
```

*Reference: [Apache Hive - Built-in functions](https://cwiki.apache.org/confluence/display/hive/languagemanual+udf#LanguageManualUDF-Built-inFunctions)*

#### Numeric functions

In [77]:
%%sparksql

SELECT  ROUND(PI(),4) AS pi,
        ROUND(LOG(2,10),2) AS log10,
        SQRT(25) AS sqrt,
        POW(10,2) AS pow

pi,log10,sqrt,pow
3.1416,3.32,5.0,100.0


In [97]:
%%sparksql

SELECT  video_id, category_id, CONCAT(video_id,'-',category_id) AS newid,
        NVL(category_id, 0) AS category_id,
        LENGTH(channel_title) AS length_title,
        CASE WHEN likes <= 20000 THEN 'group 1'
             WHEN likes > 20000 AND likes < 30000 THEN 'group 2'
             ELSE 'group 3' END AS group_like
FROM tbl_youtube
LIMIT 5

video_id,category_id,newid,category_id,length_title,group_like
2kyS6SvSYSE,22,2kyS6SvSYSE-22,22,12,group 3
1ZAPwfrtAFY,24,1ZAPwfrtAFY-24,24,15,group 3
5qpjK5DgCt4,23,5qpjK5DgCt4-23,23,12,group 3
puqaWrEC7tY,24,puqaWrEC7tY-24,24,21,group 1
d380meD0W0M,24,d380meD0W0M-24,24,8,group 3


#### Text functions

In [4]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_seafood
STORED AS PARQUET
LOCATION 'tbl_seafood'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    ('2021-10-01', 'Shrimp, frozen, chem free', 26,'  usd/kg','54HPG', 'VN'),
    ('2021-10-02', 'FISH, fresh, chemical-free', 25,'USD  /KG','9BGG', 'US'),
    ('2021-10-03', 'Prawn, frz, chemical-free', 19, 'USD/ kg','23LSN', 'UK'),
    ('2021-10-04', 'CraB, Frozen, chem free', 20, 'USD / kg','001HNI', 'NZL')
AS (date, product, price, unit, code, country)

In [5]:
%%sparksql

SELECT * from tbl_seafood

date,product,price,unit,code,country
2021-10-04,"CraB, Frozen, chem free",20,USD / kg,001HNI,NZL
2021-10-01,"Shrimp, frozen, chem free",26,usd/kg,54HPG,VN
2021-10-02,"FISH, fresh, chemical-free",25,USD /KG,9BGG,US
2021-10-03,"Prawn, frz, chemical-free",19,USD/ kg,23LSN,UK


In [59]:
%%sparksql

SELECT *, 
        LOWER(product_name) AS norm_name,
        REGEXP_REPLACE(LOWER(product_frz),'frz','frozen') AS norm_frz,
        CASE WHEN product_chem LIKE '%chem free%' 
             THEN 'chemical-free' 
             ELSE product_chem END AS norm_chem,
        SUBSTRING(norm_code,0,3) AS code_number,
        CONCAT(norm_code, '-', country) AS full_code,
        REGEXP_EXTRACT(norm_code, '[A-Z]+',0) AS code_location
FROM
    (SELECT *, 
            SPLIT(product,',')[0] AS product_name,
            SPLIT(product,',')[1] AS product_frz,
            SPLIT(product,',')[2] AS product_chem,
            LPAD(code,6,0) AS norm_code,
            REGEXP_REPLACE(UPPER(TRIM(unit)),' ','') AS norm_unit
    FROM tbl_seafood)

date,product,price,unit,code,country,product_name,product_frz,product_chem,norm_code,norm_unit,norm_name,norm_frz,norm_chem,code_number,full_code,code_location
2021-10-04,"CraB, Frozen, chem free",20,USD / kg,001HNI,NZL,CraB,Frozen,chem free,001HNI,USD/KG,crab,frozen,chemical-free,001,001HNI-NZL,HNI
2021-10-01,"Shrimp, frozen, chem free",26,usd/kg,54HPG,VN,Shrimp,frozen,chem free,054HPG,USD/KG,shrimp,frozen,chemical-free,054,054HPG-VN,HPG
2021-10-02,"FISH, fresh, chemical-free",25,USD /KG,9BGG,US,FISH,fresh,chemical-free,009BGG,USD/KG,fish,fresh,chemical-free,009,009BGG-US,BGG
2021-10-03,"Prawn, frz, chemical-free",19,USD/ kg,23LSN,UK,Prawn,frz,chemical-free,023LSN,USD/KG,prawn,frozen,chemical-free,023,023LSN-UK,LSN


# 3. Data structures

## 3.1. Array type
- Definition: same as Numpy's array
- Schema: `ARRAY<STRING>`
- Inserting: `ARRAY('hung', 'linh',...)`
- Accessing: `A[1]`, start with 0
- Techniques:
    - Unpacking: `LATERAL VIEW, EXPLODE, POSEXPLODE, INLINE,...`
    - Higher order functions: `TRANSFORM, FILTER, EXISTS, AGGREGATE` ([read more](https://databricks.com/blog/2017/05/24/working-with-nested-data-using-higher-order-functions-in-sql-on-databricks.html))
    - Basic functions: `SIZE, ARRAY_CONTAINS, SORT_ARRAY, CONCAT_WS, SEQUENCE,...` (same as in pyspark)

##### Create array

In [4]:
%%sparksql

CREATE EXTERNAL TABLE IF NOT EXISTS tbl_movie
STORED AS PARQUET
LOCATION 'tbl_movie'
TBLPROPERTIES ('parquet.compression'='snappy')

SELECT *
FROM VALUES
    (1, 'Money heist', ARRAY('Monte','Corberó', 'Alonso'), 'criminal, sex, drama', 'Portuga', 4.8, 4.6),
    (2, 'Stranger thing', ARRAY('Finn','Millie','Noah'), 'horror, science, drama, thriller', 'US', 4.9, 4.5),
    (3, 'Hill house',  ARRAY('Victoria','Kate','Oliver'), 'horror, supernature','US', 4.7, 4.8),
    (4, 'Squid game', ARRAY('jung-jae','Gong yoo','Ha-joon'), NULL, 'Korea', 4.6, 4.6)
AS (id, name, actors, genre, country, idbm_rating, tomatoes_rating)

In [5]:
%%sparksql

DESC tbl_movie

col_name,data_type,comment
id,int,null
name,string,null
actors,array<string>,null
genre,string,null
country,string,null
idbm_rating,"decimal(2,1)",null
tomatoes_rating,"decimal(2,1)",null


In [6]:
%%sparksql
SELECT *, 
SPLIT(genre,',') AS group_genre, 
ARRAY(idbm_rating, tomatoes_rating) as rating
FROM tbl_movie

id,name,actors,genre,country,idbm_rating,tomatoes_rating,group_genre,rating
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,"['criminal', ' sex', ' drama']","[Decimal('4.8'), Decimal('4.6')]"
2,Stranger thing,"['Finn', 'Millie', 'Noah']","horror, science, drama, thriller",US,4.9,4.5,"['horror', ' science', ' drama', ' thriller']","[Decimal('4.9'), Decimal('4.5')]"
3,Hill house,"['Victoria', 'Kate', 'Oliver']","horror, supernature",US,4.7,4.8,"['horror', ' supernature']","[Decimal('4.7'), Decimal('4.8')]"
4,Squid game,"['jung-jae', 'Gong yoo', 'Ha-joon']",null,Korea,4.6,4.6,null,"[Decimal('4.6'), Decimal('4.6')]"


##### Array manipulation

In [7]:
%%sparksql
SELECT *, 
actors[0] AS first_actor, 
actors[1] AS second_actor,
SLICE(actors, 1,2) AS slice_array
FROM tbl_movie

id,name,actors,genre,country,idbm_rating,tomatoes_rating,first_actor,second_actor,slice_array
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Monte,Corberó,"['Monte', 'Corberó']"
2,Stranger thing,"['Finn', 'Millie', 'Noah']","horror, science, drama, thriller",US,4.9,4.5,Finn,Millie,"['Finn', 'Millie']"
3,Hill house,"['Victoria', 'Kate', 'Oliver']","horror, supernature",US,4.7,4.8,Victoria,Kate,"['Victoria', 'Kate']"
4,Squid game,"['jung-jae', 'Gong yoo', 'Ha-joon']",null,Korea,4.6,4.6,jung-jae,Gong yoo,"['jung-jae', 'Gong yoo']"


In [8]:
%%sparksql
select * from tbl_movie LATERAL VIEW EXPLODE(actors) AS explode_actor LIMIT 3

id,name,actors,genre,country,idbm_rating,tomatoes_rating,explode_actor
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Monte
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Corberó
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Alonso


In [9]:
%%sparksql
select *, EXPLODE(actors) AS explode_actor from tbl_movie LIMIT 3

id,name,actors,genre,country,idbm_rating,tomatoes_rating,explode_actor
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Monte
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Corberó
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Alonso


In [10]:
%%sparksql
select *, POSEXPLODE(actors) AS (pos, name) from tbl_movie LIMIT 3

id,name,actors,genre,country,idbm_rating,tomatoes_rating,pos,name
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,0,Monte
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,1,Corberó
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,2,Alonso


In [11]:
%%sparksql
select *, ARRAY_JOIN(actors, "; ") AS list_actors from tbl_movie

id,name,actors,genre,country,idbm_rating,tomatoes_rating,list_actors
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,Monte; Corberó; Alonso
2,Stranger thing,"['Finn', 'Millie', 'Noah']","horror, science, drama, thriller",US,4.9,4.5,Finn; Millie; Noah
3,Hill house,"['Victoria', 'Kate', 'Oliver']","horror, supernature",US,4.7,4.8,Victoria; Kate; Oliver
4,Squid game,"['jung-jae', 'Gong yoo', 'Ha-joon']",null,Korea,4.6,4.6,jung-jae; Gong yoo; Ha-joon


In [12]:
%%sparksql

SELECT *, 
FILTER(actors, x -> x like '%oo%') AS filter_array,
CONCAT(actors, FILTER(actors, x -> x like '%oo%')) AS concat_array,
ARRAY_INTERSECT(actors, FILTER(actors, x -> x like '%oo%')) AS intersect,
ARRAY_UNION(actors, FILTER(actors, x -> x like '%oo%')) AS union,
SHUFFLE(actors) AS shuffer,
ARRAY_SORT(actors) AS sorted_array,
ARRAY_MAX(actors) AS array_max,
ARRAY_MIN(actors) AS array_min,
ARRAY_DISTINCT(actors) AS distinct_array
FROM tbl_movie

  

id,name,actors,genre,country,idbm_rating,tomatoes_rating,filter_array,concat_array,intersect,union,shuffer,sorted_array,array_max,array_min,distinct_array
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,[],"['Monte', 'Corberó', 'Alonso']",[],"['Monte', 'Corberó', 'Alonso']","['Corberó', 'Monte', 'Alonso']","['Alonso', 'Corberó', 'Monte']",Monte,Alonso,"['Monte', 'Corberó', 'Alonso']"
2,Stranger thing,"['Finn', 'Millie', 'Noah']","horror, science, drama, thriller",US,4.9,4.5,[],"['Finn', 'Millie', 'Noah']",[],"['Finn', 'Millie', 'Noah']","['Noah', 'Millie', 'Finn']","['Finn', 'Millie', 'Noah']",Noah,Finn,"['Finn', 'Millie', 'Noah']"
3,Hill house,"['Victoria', 'Kate', 'Oliver']","horror, supernature",US,4.7,4.8,[],"['Victoria', 'Kate', 'Oliver']",[],"['Victoria', 'Kate', 'Oliver']","['Oliver', 'Victoria', 'Kate']","['Kate', 'Oliver', 'Victoria']",Victoria,Kate,"['Victoria', 'Kate', 'Oliver']"
4,Squid game,"['jung-jae', 'Gong yoo', 'Ha-joon']",null,Korea,4.6,4.6,"['Gong yoo', 'Ha-joon']","['jung-jae', 'Gong yoo', 'Ha-joon', 'Gong yoo', 'Ha-joon']","['Gong yoo', 'Ha-joon']","['jung-jae', 'Gong yoo', 'Ha-joon']","['Ha-joon', 'Gong yoo', 'jung-jae']","['Gong yoo', 'Ha-joon', 'jung-jae']",jung-jae,Gong yoo,"['jung-jae', 'Gong yoo', 'Ha-joon']"


In [30]:
%%sparksql

SELECT *,
TRANSFORM(actors, actors -> CONCAT(actors,'-ss1')) as actor_ss1,
EXISTS(actors, actors -> length(actors) > 7) as actor_filter
FROM tbl_movie

id,name,actors,genre,country,idbm_rating,tomatoes_rating,actor_ss1,actor_filter
1,Money heist,"['Monte', 'Corberó', 'Alonso']","criminal, sex, drama",Portuga,4.8,4.6,"['Monte-ss1', 'Corberó-ss1', 'Alonso-ss1']",False
2,Stranger thing,"['Finn', 'Millie', 'Noah']","horror, science, drama, thriller",US,4.9,4.5,"['Finn-ss1', 'Millie-ss1', 'Noah-ss1']",False
3,Hill house,"['Victoria', 'Kate', 'Oliver']","horror, supernature",US,4.7,4.8,"['Victoria-ss1', 'Kate-ss1', 'Oliver-ss1']",True
4,Squid game,"['jung-jae', 'Gong yoo', 'Ha-joon']",null,Korea,4.6,4.6,"['jung-jae-ss1', 'Gong yoo-ss1', 'Ha-joon-ss1']",True


## 3.2. Struct type
- Definition: two or more arrays zipped together
- Schema: `ARRAY<STRUCT<id:INT, name:STRING, interest:STRING>>`
- Inserting: `ARRAY(STRUCT(0, 'hung', 'buom'), STRUCT(1, 'linh', 'chim'))`
- Accessing: `S.id, S.name, S.interest`
- Techniques: `ARRAYS_ZIP`

## 3.3. Map type
- Definition: same as Python's dict
- Schema: `MAP<STRING, STRING>`
- Inserting: `MAP('0', 'hung', '1', 'linh')`
- Accessing: `M['0']`
- Techniques:
    - `LATERAL VIEW, EXPLODE, POSEEXPLODE, INLINE,...`
    - `MAP_KEYS, MAP_VALUES, STR_TO_MAP`
    - `GET_JSON_OBJECT`